# Painel do lojista - INF 325 - Grupo 5

Este notebook cria uma base de dados para um marketplace. Aqui encontrará o modelo lógico da base de dados, representando a parte transacional do projeto.

O arquivo está organizado da seguinte forma:

- [1. Intro](#1-intro)
- [2. Criação do banco de dados](#2-criação-do-banco-de-dados)
    - [2.1. Criação das tabelas](#21-criação-das-tabelas)
- [3. Inserindo Registros](#3-inserindo-registros)
- [4. Queries usadas na feature/tela](#4-queries-usadas-na-featuretela)
    - [QUERY 1: Listar Ultimos produtos vendidos por status, paginado](#query-1-listar-ultimos-produtos-vendidos-por-status-paginado)
    - [QUERY 2: Listar Reclamações das compras, paginado](#query-2-listar-reclamações-das-compras-paginado)
    - [QUERY 3: Saldo disponível e indisponível do lojista](#query-3-saldo-disponível-e-indisponível-do-lojista)
- [5. Queries usadas p/ preencher tabelas no MongoDB (Jobs/ETLs)](#5-queries-usadas-p-preencher-tabelas-no-mongodb-jobsetls)
    - [QUERY 4: Extrai totalizadores de vendas e avaliações agrupados dia-a-dia dado um um período (Big numbers) ](#query-4-extrai-totalizadores-de-vendas-e-avaliações-agrupados-dia-a-dia-dado-um-um-período-big-numbers)
    - [QUERY 5: Extração de indicadores/recomendações para melhorias dos produtos ofertados (por perídodo)](#query-5-extração-de-indicadoresrecomendações-para-melhorias-dos-produtos-ofertados-por-perídodo)


## 1. Intro

Este notebook visa modelar a parte transacional, com vendas realizadas por um lojista, produtos, as avaliações de seus produtos e também possíveis reclamações.

O objetivo é, com base nesse modelo, criar as queries com os dados necessários para atender o protótipo proposto de um Painel do Lojista de marketplace.

Abaixo é possível visualizá-lo:
![Protótipo final](./assets/prototipo.png)
[Excalidraw Protótipos](https://excalidraw.com/#json=jKsdPqAdqJDV9dO0XwtJM,41k5DmUXu2-aMrLpYgkfFg)

## 2. Criação do banco de dados

In [ ]:
-- resolver problemas em tempo de desenvolvimento (ignore se for a primeira execução, se quiser) 
DROP DATABASE IF EXISTS `marketplace_db`;


Query OK

### 2.1 Criação da base de dados

In [ ]:
-- Criação do Banco de Dados
CREATE DATABASE IF NOT EXISTS `marketplace_db`;
USE `marketplace_db`;

### 2.1 Criação das tabelas

In [ ]:
-- -----------------------------------------------------
-- Tabela `lojistas`
-- -----------------------------------------------------
CREATE TABLE IF NOT EXISTS `lojistas` (
  `id` INT NOT NULL AUTO_INCREMENT,
  `nome` VARCHAR(255) NOT NULL,
  `email` VARCHAR(255) NOT NULL UNIQUE,
  `dados_cadastrais` VARCHAR(255) NULL,
  PRIMARY KEY (`id`));

-- -----------------------------------------------------
-- Tabela `clientes`
-- -----------------------------------------------------
CREATE TABLE IF NOT EXISTS `clientes` (
  `id` INT NOT NULL AUTO_INCREMENT,
  `nome` VARCHAR(255) NOT NULL,
  `email` VARCHAR(255) NOT NULL UNIQUE,
  `perfil` VARCHAR(45) NOT NULL DEFAULT 'padrão',
  `endereco_entrega` VARCHAR(255) NULL,
  PRIMARY KEY (`id`));

-- -----------------------------------------------------
-- Tabela `categorias`
-- -----------------------------------------------------
CREATE TABLE IF NOT EXISTS `categorias` (
  `id` INT NOT NULL AUTO_INCREMENT,
  `nome` VARCHAR(255) NOT NULL,
  PRIMARY KEY (`id`));

-- -----------------------------------------------------
-- Tabela `produtos`
-- -----------------------------------------------------
CREATE TABLE IF NOT EXISTS `produtos` (
  `id` INT NOT NULL AUTO_INCREMENT,
  `nome` VARCHAR(255) NOT NULL,
  `preco` DECIMAL(10,2) NOT NULL,
  `estoque` INT NOT NULL DEFAULT 0,
  `lojista_id` INT NOT NULL,
  `descricao` TEXT(4000) NULL,
  PRIMARY KEY (`id`),
  CONSTRAINT `fk_produtos_lojistas`
    FOREIGN KEY (`lojista_id`)
    REFERENCES `lojistas` (`id`)
);

-- -----------------------------------------------------
-- Tabela associativa `produtos_categorias`
-- -----------------------------------------------------
CREATE TABLE IF NOT EXISTS `produtos_categorias` (
  `produto_id` INT NOT NULL,
  `categoria_id` INT NOT NULL,
  PRIMARY KEY (`produto_id`, `categoria_id`),
  CONSTRAINT `fk_prodcat_produtos`
    FOREIGN KEY (`produto_id`)
    REFERENCES `produtos` (`id`),
  CONSTRAINT `fk_prodcat_categorias`
    FOREIGN KEY (`categoria_id`)
    REFERENCES `categorias` (`id`)
);

-- -----------------------------------------------------
-- Tabela `pedidos`
-- -----------------------------------------------------
CREATE TABLE IF NOT EXISTS `pedidos` (
  `id` INT NOT NULL AUTO_INCREMENT,
  `cliente_id` INT NOT NULL,
  `data` DATETIME NOT NULL,
  `status` VARCHAR(45) NOT NULL,
  PRIMARY KEY (`id`),
  CONSTRAINT `fk_pedidos_clientes`
    FOREIGN KEY (`cliente_id`)
    REFERENCES `clientes` (`id`)
);

-- -----------------------------------------------------
-- Tabela `itens_pedidos`
-- -----------------------------------------------------
CREATE TABLE IF NOT EXISTS `itens_pedidos` (
  `pedido_id` INT NOT NULL,
  `produto_id` INT NOT NULL,
  `quantidade` INT NOT NULL,
  `preco_unitario` DECIMAL(10,2) NOT NULL,
  PRIMARY KEY (`pedido_id`, `produto_id`),
  CONSTRAINT `fk_itens_pedidos`
    FOREIGN KEY (`pedido_id`)
    REFERENCES `pedidos` (`id`),
  CONSTRAINT `fk_itens_produtos`
    FOREIGN KEY (`produto_id`)
    REFERENCES `produtos` (`id`)
);

-- -----------------------------------------------------
-- Tabela `Avaliacoes`
-- -----------------------------------------------------
CREATE TABLE IF NOT EXISTS `avaliacoes` (
  `id` INT NOT NULL AUTO_INCREMENT,
  `cliente_id` INT NOT NULL,
  `produto_id` INT NOT NULL,
  `nota` INT NOT NULL,
  `comentario` TEXT NULL,
  `data` DATETIME NOT NULL,
  PRIMARY KEY (`id`),
  CONSTRAINT `fk_avaliacoes_clientes`
    FOREIGN KEY (`cliente_id`)
    REFERENCES `clientes` (`id`),
  CONSTRAINT `fk_avaliacoes_produtos`
    FOREIGN KEY (`produto_id`)
    REFERENCES `produtos` (`id`)
);

-- -----------------------------------------------------
-- Tabela `precos`
-- -----------------------------------------------------
CREATE TABLE IF NOT EXISTS `precos` (
  `id` INT NOT NULL AUTO_INCREMENT,
  `produto_id` INT NOT NULL,
  `valor` DECIMAL(10,2) NOT NULL,
  `tipo_preco` VARCHAR(45) NOT NULL,
  `data_inicio_vigencia` DATE NOT NULL,
  `data_fim_vigencia` DATE NULL,
  `ativo` BOOLEAN NOT NULL,
  `data_criacao` TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP,
  PRIMARY KEY (`id`),
  CONSTRAINT `fk_precos_produtos`
    FOREIGN KEY (`produto_id`)
    REFERENCES `produtos` (`id`)
);

-- -----------------------------------------------------
-- Tabela `reclamacoes`
-- -----------------------------------------------------
CREATE TABLE IF NOT EXISTS `reclamacoes` (
  `id` INT NOT NULL AUTO_INCREMENT,
  `pedido_id` INT NOT NULL,
  `produto_id` INT NOT NULL,
  `titulo` VARCHAR(255) NOT NULL,
  `texto` TEXT NULL,
  `data` DATETIME NOT NULL,
  PRIMARY KEY (`id`),
  CONSTRAINT `fk_reclamacoes_pedidos`
    FOREIGN KEY (`pedido_id`)
    REFERENCES `pedidos` (`id`),
  CONSTRAINT `fk_reclamacoes_produtos`
    FOREIGN KEY (`produto_id`)
    REFERENCES `produtos` (`id`)
);

-- -----------------------------------------------------
-- Tabela `transacoes`
-- -----------------------------------------------------
CREATE TABLE IF NOT EXISTS `transacoes` (
  `id` INT NOT NULL AUTO_INCREMENT,
  `lojista_id` INT NOT NULL,
  `valor` DECIMAL(10,2) NOT NULL,
  `tipo` VARCHAR(45) NOT NULL,
  `status` ENUM('Sacado', 'Disponível', 'Indisponível') NOT NULL DEFAULT 'Indisponível',
  `data` DATETIME NOT NULL,
  PRIMARY KEY (`id`),
  CONSTRAINT `fk_transacoes_lojistas`
    FOREIGN KEY (`lojista_id`)
    REFERENCES `lojistas` (`id`)
);


Query OK

## 3. Inserindo registros

In [4]:
-- Lojistas
INSERT INTO `lojistas` (`nome`, `email`, `dados_cadastrais`) VALUES
('TecnoLoja LTDA', 'contato@techcell.com', 'CNPJ: 12.345.678/0001-90'),
('Casa & Eletro Premium', 'vendas@casaeletro.com', 'CNPJ: 23.456.789/0001-01'),
('InfoWorld Solutions', 'atendimento@infoworld.com', 'CNPJ: 34.567.890/0001-12');

-- Clientes
INSERT INTO `clientes` (`nome`, `email`, `perfil`, `endereco_entrega`) VALUES
('João Silva', 'joao.silva@email.com', 'padrão', 'Rua do Beco Diagonal, 93'), 
('Maria Oliveira', 'maria.oliveira@email.com', 'vip', 'Avenida Privet Drive, 4'),
('Carlos Pereira', 'carlos.p@email.com', 'padrão', 'Rua Grimmauld Place, 12'), 
('Ana Costa', 'ana.costa@email.com', 'padrão', 'Travessa Godrics Hollow, 157'),
('Pedro Martins', 'pedro.m@email.com', 'vip', 'Alameda Hogsmeade, 234'), 
('Sandra Ferreira', 'sandra.f@email.com', 'padrão', 'Rua Knockturn Alley, 66'),
('Lucas Gomes', 'lucas.g@email.com', 'padrão', 'Avenida Spinners End, 89'), 
('Juliana Rodrigues', 'juliana.r@email.com', 'padrão', 'Rua Little Whinging, 321'),
('Gabriel Almeida', 'gabriel.a@email.com', 'vip', 'Travessa Ottery St. Catchpole, 45'), 
('Beatriz Barbosa', 'beatriz.b@email.com', 'padrão', 'Rua Shell Cottage, 178'),
('Marcos Souza', 'marcos.s@email.com', 'padrão', 'Alameda Diagon Alley, 412'), 
('Fernanda Lima', 'fernanda.l@email.com', 'padrão', 'Avenida Flourish and Blotts, 73'),
('Rafael Nunes', 'rafael.n@email.com', 'padrão', 'Rua Olivanders, 298'), 
('Camila Santos', 'camila.s@email.com', 'vip', 'Travessa Gringotts, 156'),
('Tiago Mendes', 'tiago.m@email.com', 'padrão', 'Rua Leaky Cauldron, 507'), 
('Larissa Castro', 'larissa.c@email.com', 'padrão', 'Alameda Madam Malkins, 84'),
('Bruno Ribeiro', 'bruno.r@email.com', 'padrão', 'Avenida Quality Quidditch, 129'), 
('Patrícia Rocha', 'patricia.r@email.com', 'padrão', 'Rua Weasleys Wizard Wheezes, 365'),
('Felipe Jesus', 'felipe.j@email.com', 'padrão', 'Travessa Borgin and Burkes, 241'), 
('Amanda Dias', 'amanda.d@email.com', 'vip', 'Rua Three Broomsticks, 192');

-- Categorias (3 registros)
INSERT INTO `categorias` (`nome`) VALUES
('Eletrônicos'),
('Eletrodomésticos'),
('Informática');

-- Produtos
-- Lojista 1
INSERT INTO `produtos` (`nome`, `preco`, `estoque`, `lojista_id`, `descricao`) VALUES
('Galaxy S24 Ultra 512GB', 5899.90, 25, 1, 'Smartphone premium com câmera de 200MP e S Pen integrada'),
('iPhone 15 Pro Max 512GB', 8999.00, 18, 1, 'iPhone mais avançado com chip A17 Pro e titânio'),
('Xiaomi 14 Pro 256GB', 3299.90, 40, 1, 'Smartphone com Snapdragon 8 Gen 3 e câmera Leica'),
('MacBook Air M3 16GB', 12999.00, 15, 1, 'Notebook ultraportátil com chip M3 e tela Liquid Retina'),
('Desktop Gamer RTX 4080', 8499.90, 8, 1, 'PC Gamer completo com RTX 4080 e Ryzen 7 7800X3D');

-- Lojista 2
INSERT INTO `produtos` (`nome`, `preco`, `estoque`, `lojista_id`, `descricao`) VALUES
('Galaxy S24 Ultra 512GB', 4499.90, 20, 2, 'Versão premium do Galaxy S24 Ultra com mais armazenamento'),
('iPhone 15 Pro 256GB', 7999.00, 22, 2, 'iPhone 15 Pro com tecnologia Pro avançada'),
('Geladeira Side by Side 540L', 4299.00, 12, 2, 'Geladeira com freezer lateral e dispenser de água'),
('Lava e Seca 12kg Inverter', 3899.90, 10, 2, 'Lava e seca com tecnologia inverter e 15 programas'),
('Micro-ondas 32L Inox', 899.90, 35, 2, 'Micro-ondas com grill e painel digital touch'),
('Purificador de Ar HEPA', 1299.90, 25, 2, 'Purificador com filtro HEPA e ionizador');

-- Lojista 3
INSERT INTO `produtos` (`nome`, `preco`, `estoque`, `lojista_id`, `descricao`) VALUES
('Galaxy S24 Ultra 512GB', 4299.90, 30, 3, 'Galaxy S24 Plus com tela de 6.7 polegadas e câmera avançada'),
('Xiaomi 14 Ultra 512GB', 4199.90, 28, 3, 'Xiaomi 14 Ultra com câmera profissional Leica'),
('Monitor Gamer 27" 240Hz', 2499.90, 20, 3, 'Monitor QHD com alta taxa de atualização para gamers'),
('Headset Gamer Wireless', 899.90, 45, 3, 'Headset sem fio com som surround 7.1 e RGB'),
('Teclado Mecânico RGB', 599.90, 60, 3, 'Teclado mecânico com switches Cherry MX e iluminação RGB'),
('Mouse Gamer 16000 DPI', 399.90, 80, 3, 'Mouse para jogos com sensor óptico de alta precisão');

-- produtos_categorias
INSERT INTO `produtos_categorias` (`produto_id`, `categoria_id`) VALUES
-- Produtos do Lojista 1
(1, 1),
(2, 1),
(3, 1),
(4, 3),
(5, 3),
-- Produtos do Lojista 2
(6, 1),
(7, 1),
(8, 2),
(9, 2),
(10, 2),
(11, 2),
-- Produtos do Lojista 3
(12, 1),
(13, 1),
(14, 3),
(15, 3),
(16, 3),
(17, 3);

-- Pedidos com status "Entregue"
INSERT INTO `pedidos` (`cliente_id`, `data`, `status`) VALUES
(1, '2025-07-01 14:30:00', 'Entregue');
INSERT INTO `itens_pedidos` (`pedido_id`, `produto_id`, `quantidade`, `preco_unitario`) VALUES
(1, 1, 1, 4899.90);

INSERT INTO `pedidos` (`cliente_id`, `data`, `status`) VALUES
(2, '2025-07-02 10:15:00', 'Entregue');
INSERT INTO `itens_pedidos` (`pedido_id`, `produto_id`, `quantidade`, `preco_unitario`) VALUES
(2, 2, 1, 8999.00);

INSERT INTO `pedidos` (`cliente_id`, `data`, `status`) VALUES
(3, '2025-07-03 16:45:00', 'Entregue');
INSERT INTO `itens_pedidos` (`pedido_id`, `produto_id`, `quantidade`, `preco_unitario`) VALUES
(3, 3, 1, 3299.90);

INSERT INTO `pedidos` (`cliente_id`, `data`, `status`) VALUES
(4, '2025-07-04 09:20:00', 'Entregue');
INSERT INTO `itens_pedidos` (`pedido_id`, `produto_id`, `quantidade`, `preco_unitario`) VALUES
(4, 4, 1, 12999.00);

INSERT INTO `pedidos` (`cliente_id`, `data`, `status`) VALUES
(5, '2025-07-05 13:10:00', 'Entregue');
INSERT INTO `itens_pedidos` (`pedido_id`, `produto_id`, `quantidade`, `preco_unitario`) VALUES
(5, 5, 1, 8499.90);

INSERT INTO `pedidos` (`cliente_id`, `data`, `status`) VALUES
(6, '2025-07-06 11:00:00', 'Entregue');
INSERT INTO `itens_pedidos` (`pedido_id`, `produto_id`, `quantidade`, `preco_unitario`) VALUES
(6, 6, 1, 5499.90);

INSERT INTO `pedidos` (`cliente_id`, `data`, `status`) VALUES
(7, '2025-07-07 15:30:00', 'Entregue');
INSERT INTO `itens_pedidos` (`pedido_id`, `produto_id`, `quantidade`, `preco_unitario`) VALUES
(7, 7, 1, 7999.00);

INSERT INTO `pedidos` (`cliente_id`, `data`, `status`) VALUES
(8, '2025-07-08 08:45:00', 'Entregue');
INSERT INTO `itens_pedidos` (`pedido_id`, `produto_id`, `quantidade`, `preco_unitario`) VALUES
(8, 8, 1, 4299.00);

INSERT INTO `pedidos` (`cliente_id`, `data`, `status`) VALUES
(9, '2025-07-09 17:20:00', 'Entregue');
INSERT INTO `itens_pedidos` (`pedido_id`, `produto_id`, `quantidade`, `preco_unitario`) VALUES
(9, 9, 1, 3899.90);

INSERT INTO `pedidos` (`cliente_id`, `data`, `status`) VALUES
(10, '2025-07-10 12:15:00', 'Entregue');
INSERT INTO `itens_pedidos` (`pedido_id`, `produto_id`, `quantidade`, `preco_unitario`) VALUES
(10, 10, 1, 899.90);

INSERT INTO `pedidos` (`cliente_id`, `data`, `status`) VALUES
(11, '2025-07-11 14:50:00', 'Entregue');
INSERT INTO `itens_pedidos` (`pedido_id`, `produto_id`, `quantidade`, `preco_unitario`) VALUES
(11, 11, 1, 1299.90);

INSERT INTO `pedidos` (`cliente_id`, `data`, `status`) VALUES
(12, '2025-07-12 10:30:00', 'Entregue');
INSERT INTO `itens_pedidos` (`pedido_id`, `produto_id`, `quantidade`, `preco_unitario`) VALUES
(12, 12, 1, 4299.90);

INSERT INTO `pedidos` (`cliente_id`, `data`, `status`) VALUES
(13, '2025-07-13 16:00:00', 'Entregue');
INSERT INTO `itens_pedidos` (`pedido_id`, `produto_id`, `quantidade`, `preco_unitario`) VALUES
(13, 13, 1, 4199.90);

INSERT INTO `pedidos` (`cliente_id`, `data`, `status`) VALUES
(14, '2025-07-14 09:40:00', 'Entregue');
INSERT INTO `itens_pedidos` (`pedido_id`, `produto_id`, `quantidade`, `preco_unitario`) VALUES
(14, 14, 1, 2499.90);

INSERT INTO `pedidos` (`cliente_id`, `data`, `status`) VALUES
(15, '2025-07-15 13:25:00', 'Entregue');
INSERT INTO `itens_pedidos` (`pedido_id`, `produto_id`, `quantidade`, `preco_unitario`) VALUES
(15, 15, 1, 899.90);

INSERT INTO `pedidos` (`cliente_id`, `data`, `status`) VALUES
(16, '2025-07-16 11:10:00', 'Entregue');
INSERT INTO `itens_pedidos` (`pedido_id`, `produto_id`, `quantidade`, `preco_unitario`) VALUES
(16, 16, 1, 599.90);

INSERT INTO `pedidos` (`cliente_id`, `data`, `status`) VALUES
(17, '2025-07-17 15:55:00', 'Entregue');
INSERT INTO `itens_pedidos` (`pedido_id`, `produto_id`, `quantidade`, `preco_unitario`) VALUES
(17, 17, 1, 399.90);

INSERT INTO `pedidos` (`cliente_id`, `data`, `status`) VALUES
(18, '2025-07-18 09:30:00', 'Processando'),
(19, '2025-07-18 14:20:00', 'Enviado'),
(20, '2025-07-19 10:15:00', 'Cancelado'),
(1, '2025-07-19 16:40:00', 'Processando'),
(2, '2025-07-20 08:25:00', 'Enviado'),
(3, '2025-07-20 13:50:00', 'Processando'),
(4, '2025-07-21 11:30:00', 'Enviado'),
(5, '2025-07-21 17:10:00', 'Cancelado'),
(6, '2025-07-22 09:45:00', 'Processando'),
(7, '2025-07-22 14:35:00', 'Enviado');

INSERT INTO `itens_pedidos` (`pedido_id`, `produto_id`, `quantidade`, `preco_unitario`) VALUES
(18, 3, 1, 3299.90),
(19, 6, 1, 5499.90),
(20, 14, 1, 2499.90),
(21, 7, 1, 7999.00),
(22, 11, 1, 1299.90),
(23, 16, 2, 599.90),
(24, 8, 1, 4299.00),
(25, 13, 1, 4199.90),
(26, 15, 1, 899.90),
(27, 9, 1, 3899.90);

-- Transações
INSERT INTO `transacoes` (`lojista_id`, `valor`, `tipo`, `status`, `data`) VALUES
(1, 4899.90, 'Venda', 'Sacado', '2025-07-01 14:35:00'),
(1, 8999.00, 'Venda', 'Sacado', '2025-07-02 10:20:00'),
(1, 3299.90, 'Venda', 'Sacado', '2025-07-03 16:50:00'),
(1, 12999.00, 'Venda', 'Sacado', '2025-07-04 09:25:00'),
(1, 8499.90, 'Venda', 'Sacado', '2025-07-05 13:15:00'),
(2, 5499.90, 'Venda', 'Sacado', '2025-07-06 11:05:00'),
(2, 7999.00, 'Venda', 'Sacado', '2025-07-07 15:35:00'),
(2, 4299.00, 'Venda', 'Sacado', '2025-07-08 08:50:00'),
(2, 3899.90, 'Venda', 'Sacado', '2025-07-09 17:25:00'),
(2, 899.90, 'Venda', 'Disponível', '2025-07-10 12:20:00'),
(2, 1299.90, 'Venda', 'Disponível', '2025-07-11 14:55:00'),
(3, 4299.90, 'Venda', 'Disponível', '2025-07-12 10:35:00'),
(3, 4199.90, 'Venda', 'Disponível', '2025-07-13 16:05:00'),
(3, 2499.90, 'Venda', 'Disponível', '2025-07-14 09:45:00'),
(3, 899.90, 'Venda', 'Disponível', '2025-07-15 13:30:00'),
(3, 599.90, 'Venda', 'Disponível', '2025-07-16 11:15:00'),
(3, 399.90, 'Venda', 'Disponível', '2025-07-17 16:00:00'),
(1, 3299.90, 'Venda', 'Disponível', '2025-07-18 09:35:00'),
(2, 5499.90, 'Venda', 'Indisponível', '2025-07-18 14:25:00'),
(3, 2499.90, 'Venda', 'Indisponível', '2025-07-19 10:20:00'),
(2, 7999.00, 'Venda', 'Indisponível', '2025-07-19 16:45:00'),
(2, 1299.90, 'Venda', 'Indisponível', '2025-07-20 08:30:00'),
(3, 1199.80, 'Venda', 'Indisponível', '2025-07-20 13:55:00'),
(2, 4299.00, 'Venda', 'Indisponível', '2025-07-21 11:35:00'),
(3, 4199.90, 'Venda', 'Indisponível', '2025-07-21 17:15:00'),
(3, 899.90, 'Venda', 'Indisponível', '2025-07-22 09:50:00'),
(2, 3899.90, 'Venda', 'Indisponível', '2025-07-22 14:40:00');

-- Avaliações para produtos entregues
-- Lojista 1
INSERT INTO `avaliacoes` (`cliente_id`, `produto_id`, `nota`, `comentario`, `data`) VALUES
(1, 1, 2, 'Produto chegou com arranhões e a bateria não dura nem meio dia. Muito decepcionante.', '2025-07-03 14:30:00'),
(2, 2, 1, 'Péssima experiência! O celular travou no primeiro dia e o vendedor não responde.', '2025-07-04 10:15:00'),
(3, 3, 2, 'Qualidade muito abaixo do esperado. A câmera não funciona direito e esquenta demais.', '2025-07-05 16:45:00'),
(4, 4, 1, 'Notebook veio com defeito na tela e demora muito para ligar. Não recomendo.', '2025-07-06 09:20:00'),
(5, 5, 2, 'PC gamer que não roda nem jogos básicos. Propaganda enganosa total.', '2025-07-07 13:10:00');

-- Lojista 2
INSERT INTO `avaliacoes` (`cliente_id`, `produto_id`, `nota`, `comentario`, `data`) VALUES
(6, 6, 5, 'Produto incrível! Superou todas as expectativas, funciona perfeitamente.', '2025-07-08 11:00:00'),
(7, 7, 4, 'Muito bom! Chegou rapidinho e exatamente como descrito. Recomendo.', '2025-07-09 15:30:00'),
(8, 8, 5, 'Geladeira excelente! Silenciosa e muito espaçosa. Atendimento nota 10.', '2025-07-10 08:45:00'),
(9, 9, 4, 'Ótima máquina! Lava e seca muito bem, programas variados. Estou satisfeito.', '2025-07-11 17:20:00'),
(10, 10, 5, 'Micro-ondas fantástico! Funcionalidades excelentes e design bonito.', '2025-07-12 12:15:00'),
(11, 11, 4, 'Purificador muito bom! Já notei diferença na qualidade do ar em casa.', '2025-07-13 14:50:00');

-- Lojista 3
INSERT INTO `avaliacoes` (`cliente_id`, `produto_id`, `nota`, `comentario`, `data`) VALUES
(12, 12, 4, 'Ótimo celular! Câmera incrível e performance excelente. Recomendo muito.', '2025-07-14 10:30:00'),
(13, 13, 5, 'Smartphone perfeito! Todas as funcionalidades funcionam perfeitamente.', '2025-07-15 16:00:00'),
(14, 14, 5, 'Monitor sensacional para jogos! Cores vibrantes e sem delay. Adorei.', '2025-07-16 09:40:00'),
(15, 15, 4, 'Headset muito bom! Som claro e confortável para longas sessões de jogo.', '2025-07-17 13:25:00'),
(16, 16, 5, 'Teclado excelente! Teclas responsivas e iluminação linda. Vale cada centavo.', '2025-07-18 11:10:00'),
(17, 17, 4, 'Mouse preciso e confortável! Perfeito para jogos competitivos.', '2025-07-19 15:55:00');

-- Reclamações apenas para produtos do Lojista 1
INSERT INTO `reclamacoes` (`pedido_id`, `produto_id`, `titulo`, `texto`, `data`) VALUES
(1, 1, 'Produto com defeito de fábrica', 'O aparelho chegou com a tela trincada e arranhões na carcaça. Inaceitável pelo preço pago.', '2025-07-02 14:30:00'),
(2, 2, 'Produto não funciona corretamente', 'O celular trava constantemente e não consegue nem fazer ligações direito. Quero meu dinheiro de volta.', '2025-07-03 10:15:00'),
(3, 3, 'Qualidade inferior ao anunciado', 'A câmera não funciona e o aparelho esquenta muito. Nada do que foi prometido na descrição.', '2025-07-04 16:45:00'),
(4, 4, 'Notebook com problemas graves', 'Tela com defeito, demora para ligar e trava durante o uso. Produto de péssima qualidade.', '2025-07-05 09:20:00'),
(5, 5, 'PC gamer não roda jogos', 'Propaganda enganosa! O computador não consegue rodar nem jogos básicos. Performance ridícula.', '2025-07-06 13:10:00');

-- Preços
INSERT INTO `precos` (`produto_id`, `valor`, `tipo_preco`, `data_inicio_vigencia`, `ativo`) VALUES
(1, 4899.90, 'Base', '2025-06-01', TRUE),
(2, 8999.00, 'Base', '2025-06-01', TRUE),
(3, 3299.90, 'Base', '2025-06-01', TRUE),
(4, 12999.00, 'Base', '2025-06-01', TRUE),
(5, 8499.90, 'Base', '2025-06-01', TRUE),
(6, 5499.90, 'Base', '2025-06-01', TRUE),
(7, 7999.00, 'Base', '2025-06-01', TRUE),
(8, 4299.00, 'Base', '2025-06-01', TRUE),
(9, 3899.90, 'Base', '2025-06-01', TRUE),
(10, 899.90, 'Base', '2025-06-01', TRUE),
(11, 1299.90, 'Base', '2025-06-01', TRUE),
(12, 4299.90, 'Base', '2025-06-01', TRUE),
(13, 4199.90, 'Base', '2025-06-01', TRUE),
(14, 2499.90, 'Base', '2025-06-01', TRUE),
(15, 899.90, 'Base', '2025-06-01', TRUE),
(16, 599.90, 'Base', '2025-06-01', TRUE),
(17, 399.90, 'Base', '2025-06-01', TRUE);

Query OK

## 4. Queries usadas na feature/tela

### QUERY 1: Listar Ultimos produtos vendidos por status, paginado

In [5]:
SELECT
    ped.data, p.nome, ped.id, ped.status, ip.preco_unitario, ip.quantidade, c.endereco_entrega
FROM
    lojistas l
JOIN produtos p ON l.id = p.lojista_id
JOIN itens_pedidos ip ON p.id = ip.produto_id
JOIN pedidos ped on ped.id = ip.pedido_id
JOIN clientes c on c.id = ped.cliente_id
WHERE 
    ped.status IN ('Processando', 'Cancelado', 'Entregue', 'Enviado')
    AND l.id = 1 -- Dinâmico pela sessão do usuário
ORDER BY
    ped.data DESC
limit 0,10;

data,nome,id,status,preco_unitario,quantidade,endereco_entrega
2025-07-18 09:30:00,Xiaomi 14 Pro 256GB,18,Processando,3299.90,1,"Rua Weasleys Wizard Wheezes, 365"
2025-07-05 13:10:00,Desktop Gamer RTX 4080,5,Entregue,8499.90,1,"Alameda Hogsmeade, 234"
2025-07-04 09:20:00,MacBook Air M3 16GB,4,Entregue,12999.00,1,"Travessa Godrics Hollow, 157"
2025-07-03 16:45:00,Xiaomi 14 Pro 256GB,3,Entregue,3299.90,1,"Rua Grimmauld Place, 12"
2025-07-02 10:15:00,iPhone 15 Pro Max 512GB,2,Entregue,8999.00,1,"Avenida Privet Drive, 4"
2025-07-01 14:30:00,Galaxy S24 Ultra 512GB,1,Entregue,4899.90,1,"Rua do Beco Diagonal, 93"


### QUERY 2: Listar Reclamações das compras, paginado

In [6]:
SELECT
    l.nome lojista, c.id cliente_id, CONCAT('Sr(a). ', SUBSTRING_INDEX(c.nome, ' ', -1)) cliente_ultimo_nome, p.nome produto, r.titulo, r.texto, r.data
FROM
    lojistas l
JOIN produtos p ON l.id = p.lojista_id
JOIN itens_pedidos ip ON p.id = ip.produto_id
JOIN pedidos ped on ped.id = ip.pedido_id
JOIN clientes c on c.id = ped.cliente_id
JOIN reclamacoes r on r.pedido_id = ip.pedido_id and r.produto_id = ip.produto_id
WHERE 
    ped.status IN ('Entregue') 
    AND l.id = 1 -- Dinâmico pela sessão do usuário
ORDER BY
    ped.data DESC
limit 0,10;

lojista,cliente_id,cliente_ultimo_nome,produto,titulo,texto,data
TecnoLoja LTDA,5,Sr(a). Martins,Desktop Gamer RTX 4080,PC gamer não roda jogos,Propaganda enganosa! O computador não consegue rodar nem jogos básicos. Performance ridícula.,2025-07-06 13:10:00
TecnoLoja LTDA,4,Sr(a). Costa,MacBook Air M3 16GB,Notebook com problemas graves,"Tela com defeito, demora para ligar e trava durante o uso. Produto de péssima qualidade.",2025-07-05 09:20:00
TecnoLoja LTDA,3,Sr(a). Pereira,Xiaomi 14 Pro 256GB,Qualidade inferior ao anunciado,A câmera não funciona e o aparelho esquenta muito. Nada do que foi prometido na descrição.,2025-07-04 16:45:00
TecnoLoja LTDA,2,Sr(a). Oliveira,iPhone 15 Pro Max 512GB,Produto não funciona corretamente,O celular trava constantemente e não consegue nem fazer ligações direito. Quero meu dinheiro de volta.,2025-07-03 10:15:00
TecnoLoja LTDA,1,Sr(a). Silva,Galaxy S24 Ultra 512GB,Produto com defeito de fábrica,O aparelho chegou com a tela trincada e arranhões na carcaça. Inaceitável pelo preço pago.,2025-07-02 14:30:00


### QUERY 3: Saldo disponível e indisponível do lojista

In [7]:
SELECT t.status, SUM(t.valor) as total
FROM transacoes t
JOIN lojistas l on l.id = t.lojista_id
WHERE 
    t.status != 'Sacado'
    AND l.id = 2 -- Dinâmico pela sessão do usuário
GROUP BY t.status;

status,total
Disponível,2199.80
Indisponível,22997.70


## 5. Queries usadas p/ preencher tabelas no MongoDB (Jobs/ETLs)

**Premissa importante:** o mariadb deve ter uma replica readonly para execução dessas extrações, evitando impactar a base transacional.

### QUERY 4: Extrai totalizadores de vendas e avaliações agrupados dia-a-dia dado um um período (Big numbers) 

In [8]:
SELECT
	data_execucao,
	lojista_id,
	lojista_nome,
	tipo,
	quantidade
FROM
	(
	SELECT
		ped.data data_execucao,
		l.id lojista_id,
		l.nome lojista_nome,
		'VENDAS' tipo,
		count(ped.id) quantidade
	FROM
		lojistas l
	JOIN produtos p ON
		l.id = p.lojista_id
	JOIN itens_pedidos ip ON
		p.id = ip.produto_id
	JOIN pedidos ped ON
		ped.id = ip.pedido_id
	WHERE
		ped.status IN ('Processando', 'Cancelado', 'Entregue', 'Enviado')
		AND ped.data BETWEEN "2025-06-01" AND "2025-07-30" -- Dinâmico, com base no período requisitado na extração
	GROUP BY
		ped.data,
		l.id,
		l.nome
UNION
	SELECT
		a.data,
		l.id lojista_id,
		l.nome lojista_nome,
		'AVALIACOES' tipo,
		count(a.id) quantidade
	FROM
		lojistas l
	JOIN produtos p ON
		l.id = p.lojista_id
	JOIN avaliacoes a ON
		a.produto_id = p.id
	WHERE
		a.data BETWEEN "2025-06-01" AND "2025-07-30" -- Dinâmico, com base no período requisitado na extração
	GROUP BY
		a.data,
		l.id,
		l.nome
) contadores_diarios
ORDER BY
	data_execucao ASC;

data_execucao,lojista_id,lojista_nome,tipo,quantidade
2025-07-01 14:30:00,1,TecnoLoja LTDA,VENDAS,1
2025-07-02 10:15:00,1,TecnoLoja LTDA,VENDAS,1
2025-07-03 14:30:00,1,TecnoLoja LTDA,AVALIACOES,1
2025-07-03 16:45:00,1,TecnoLoja LTDA,VENDAS,1
2025-07-04 09:20:00,1,TecnoLoja LTDA,VENDAS,1
2025-07-04 10:15:00,1,TecnoLoja LTDA,AVALIACOES,1
2025-07-05 13:10:00,1,TecnoLoja LTDA,VENDAS,1
2025-07-05 16:45:00,1,TecnoLoja LTDA,AVALIACOES,1
2025-07-06 09:20:00,1,TecnoLoja LTDA,AVALIACOES,1
2025-07-06 11:00:00,2,Casa & Eletro Premium,VENDAS,1


### QUERY 5: Extração de indicadores/recomendações para melhorias dos produtos ofertados (por perídodo)

Esta query extrai aspectos levantados na base transacional e expõe no mongo. Serão usados para criar recomendações para o lojista melhorar seus produtos.

Os aspectos são:
- **Boa descrição:** avaliado pela qtd de caracteres. O intuito é sugerir que o lojista descreva bem o item. (Uma descrição longa e bem escrita auxilia na decisão de compra - Premissa de que deve ser maior que 55 caracteres para poupar o tamanho dos inserts);
- **Qtd de reclamações por x vendas:** Deveria possuir uma taxa de reclamação x venda igual ou inferior a 10%;
- **Média de avaliação do produto:** É recomendado manter uma nota igual ou superior a 4.

- **Executar avaliando um período, toda madrugada**.

In [9]:
SELECT
	l.id lojista_id,
	l.nome lojista_nome,
	p.id produto_id,
	p.nome produto_nome,
	LENGTH(p.descricao) tamanho_descricao,
	LENGTH(p.descricao) > 55 AS tamanho_descricao_recomendado,
	(
	SELECT
		avg(a.nota)
	FROM
		avaliacoes a
	WHERE
		a.produto_id = p.id) AS media_avaliacoes,
	ROUND(IFNULL(
(
    SELECT count(r.id) 
    FROM reclamacoes r 
    JOIN itens_pedidos ip1 ON ip1.pedido_id = r.pedido_id
    JOIN pedidos ped1 ON ped1.id = ip1.pedido_id
    WHERE ip1.produto_id = p.id 
    AND r.data BETWEEN "2025-06-01" AND "2025-07-30" -- Dinâmico, com base no período requisitado na extração
) / (
    SELECT count(1) FROM itens_pedidos ip2
    JOIN pedidos ped2 ON ped2.id = ip2.pedido_id
    WHERE ip2.produto_id = p.id
    AND ped2.data BETWEEN "2025-06-01" AND "2025-07-30" -- Dinâmico, com base no período requisitado na extração
) * 100, 0)) as taxa_reclamacao_por_qtd_vendas,
0 AS revisado
FROM
	lojistas l
JOIN produtos p ON
	l.id = p.lojista_id;

lojista_id,lojista_nome,produto_id,produto_nome,tamanho_descricao,tamanho_descricao_recomendado,media_avaliacoes,taxa_reclamacao_por_qtd_vendas,revisado
1,TecnoLoja LTDA,1,Galaxy S24 Ultra 512GB,57,1,2.0000,100,0
1,TecnoLoja LTDA,2,iPhone 15 Pro Max 512GB,49,0,1.0000,100,0
1,TecnoLoja LTDA,3,Xiaomi 14 Pro 256GB,49,0,2.0000,50,0
1,TecnoLoja LTDA,4,MacBook Air M3 16GB,56,1,1.0000,100,0
1,TecnoLoja LTDA,5,Desktop Gamer RTX 4080,48,0,2.0000,100,0
2,Casa & Eletro Premium,6,Galaxy S24 Ultra 512GB,58,1,5.0000,0,0
2,Casa & Eletro Premium,7,iPhone 15 Pro 256GB,42,0,4.0000,0,0
2,Casa & Eletro Premium,8,Geladeira Side by Side 540L,50,0,5.0000,0,0
2,Casa & Eletro Premium,9,Lava e Seca 12kg Inverter,50,0,4.0000,0,0
2,Casa & Eletro Premium,10,Micro-ondas 32L Inox,44,0,5.0000,0,0
